In [1]:
%cd /workspace
import sys
import os

# Set PYTHONPATH environment variable for the kernel
robofin_path = os.path.join(os.getcwd(), 'robofin')
current_pythonpath = os.environ.get('PYTHONPATH', '')
if robofin_path not in current_pythonpath:
    os.environ['PYTHONPATH'] = f"{robofin_path}:{current_pythonpath}" if current_pythonpath else robofin_path

# Also add to sys.path for immediate effect
if robofin_path not in sys.path:
    sys.path.insert(0, robofin_path)

/workspace


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
def get_type(obj):
    """
    Function for displaying nested types.
    
    e.g. get_type(dict_str_float) -> "dict[str, float]"
    """
    if isinstance(obj, dict):
        if not obj:
            return "dict[?, ?]"
        key_types = {get_type(k) for k in obj.keys()}
        value_types = {get_type(v) for v in obj.values()}
        return f"dict[{', '.join(key_types)}, {', '.join(value_types)}]"
    elif isinstance(obj, list):
        if not obj:
            return "list[?]"
        elem_types = {get_type(elem) for elem in obj}
        return f"list[{', '.join(elem_types)}]"
    elif isinstance(obj, tuple):
        if not obj:
            return "tuple[?]"
        elem_types = [get_type(elem) for elem in obj]
        return f"tuple[{', '.join(elem_types)}]"
    elif isinstance(obj, set):
        if not obj:
            return "set[?]"
        elem_types = {get_type(elem) for elem in obj}
        return f"set[{', '.join(elem_types)}]"
    else:
        return type(obj).__name__

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import torch
from robofin.robots import Robot

# Load the Robot class with the standard URDF file (that uses relative filepaths)
robot = Robot("assets/panda/panda.urdf")
# robot = Robot("assets/gp7/gp7.urdf")

In [6]:
import sys
import os

# Add the pxr USD Python bindings to the Python path if they exist
usd_python_path = "/opt/PixarAnimationStudios/USD/lib/python"
if os.path.isdir(usd_python_path):
    if usd_python_path not in sys.path:
        sys.path.insert(0, usd_python_path)

from pxr import UsdGeom



## Sampler testing

In [5]:
from robofin.old.kinematics.numba import franka_arm_link_fk, franka_arm_visual_fk, franka_eef_visual_fk
from robofin.old.kinematics.torch import franka_arm_link_fk as torch_franka_arm_link_fk
from robofin.old.kinematics.torch import franka_arm_visual_fk as torch_franka_arm_visual_fk

fk = robot.fk(robot.neutral_config)

prismatic_joint = robot.auxiliary_joint_defaults["panda_finger_joint1"]
base_pose = np.eye(4)
franka_fk = franka_arm_link_fk(robot.neutral_config, prismatic_joint, base_pose)
franka_fk_dict = {
    "panda_link0": franka_fk[0],
    "panda_link1": franka_fk[1],
    "panda_link2": franka_fk[2],
    "panda_link3": franka_fk[3],
    "panda_link4": franka_fk[4],
    "panda_link5": franka_fk[5],
    "panda_link6": franka_fk[6],
    "panda_link7": franka_fk[7],
    "panda_link8": franka_fk[8],
    "panda_hand": franka_fk[9],
    "panda_grasptarget": franka_fk[10],
    "right_gripper": franka_fk[11],
    "panda_leftfinger": franka_fk[12],
    "panda_rightfinger": franka_fk[13],
}

torch_cfg = torch.Tensor(robot.neutral_config).unsqueeze(0)
torch_fk = robot.fk_torch(torch_cfg)

torch_base_pose = torch.Tensor(base_pose)
torch_franka_fk = torch_franka_arm_link_fk(torch_cfg, prismatic_joint, torch_base_pose)
torch_franka_fk_dict = {
    "panda_link0": torch_franka_fk[:,0],
    "panda_link1": torch_franka_fk[:,1],
    "panda_link2": torch_franka_fk[:,2],
    "panda_link3": torch_franka_fk[:,3],
    "panda_link4": torch_franka_fk[:,4],
    "panda_link5": torch_franka_fk[:,5],
    "panda_link6": torch_franka_fk[:,6],
    "panda_link7": torch_franka_fk[:,7],
    "panda_link8": torch_franka_fk[:,8],
    "panda_hand": torch_franka_fk[:,9],
    "panda_grasptarget": torch_franka_fk[:,10],
    "right_gripper": torch_franka_fk[:,11],
    "panda_leftfinger": torch_franka_fk[:,12],
    "panda_rightfinger": torch_franka_fk[:,13],
}

print(torch_fk["panda_hand"].shape)
print(torch_franka_fk_dict["panda_hand"].shape)


torch.Size([1, 4, 4])
torch.Size([1, 4, 4])


In [6]:
from robofin.samplers import NumpyRobotSampler, TorchRobotSampler
from robofin.old.samplers import NumpyFrankaSampler, TorchFrankaSampler

np_sampler = NumpyRobotSampler(robot)
torch_sampler = TorchRobotSampler(robot)

np_franka_sampler = NumpyFrankaSampler()
torch_franka_sampler = TorchFrankaSampler()

In [7]:
franka_ee_pts = np_franka_sampler.sample_end_effector(fk[robot.tcp_link_name].squeeze(), prismatic_joint)
ee_pts = np_sampler.sample_end_effector(fk[robot.tcp_link_name].squeeze())

print("---" * 10)
print(f"Franka EE points shape: {franka_ee_pts.shape}")
print(f"EE points shape: {ee_pts.shape}")

torch_franka_ee_pts = torch_franka_sampler.sample_end_effector(torch_franka_fk_dict[robot.tcp_link_name], prismatic_joint)
torch_ee_pts = torch_sampler.sample_end_effector(torch_fk[robot.tcp_link_name])

print("---" * 10)
print(f"Torch Franka EE points shape: {torch_franka_ee_pts.shape}")
print(f"Torch EE points shape: {torch_ee_pts.shape}")

------------------------------
Franka EE points shape: (128, 4)
EE points shape: (128, 4)
------------------------------
Torch Franka EE points shape: torch.Size([1, 128, 4])
Torch EE points shape: torch.Size([1, 128, 4])


In [8]:
def has_point(
    point_cloud: np.ndarray, point: np.ndarray, tolerance: float = 1e-6
) -> tuple[bool, np.ndarray]:
    """
    Returns:
    exists, idx (bool, np.ndarray): Whether `point` exists in the point cloud
        and the indices in the point cloud array where the point exists.
    """
    mask = np.max(np.abs(point_cloud - point), axis=1) <= tolerance
    exists = mask.any()
    idx = np.flatnonzero(mask)
    return exists, idx


def compare_point_clouds(
    pc1: np.ndarray, pc2: np.ndarray, abs_tol: float = 1e-7
) -> bool:
    """
    Return True if input point clouds are identical (within given tolerance)
    """
    # return np.allclose(pc1, pc2, atol=abs_tol)
    if np.allclose(pc1, pc2, atol=abs_tol):
        return True

    print("compare_point_clouds() returning False")
    print("Largest error:", np.abs(pc1 - pc2).max())

    return False

In [9]:
from termcolor import cprint
from robofin.samplers import get_points_on_robot_eef
from robofin.old.kinematics.numba import get_points_on_franka_eef, eef_pose_to_link8, franka_eef_link_fk, franka_arm_visual_fk

np.set_printoptions(precision=3, suppress=True)

# frame = robot.tcp_link_name # right_gripper
frame = "panda_hand"
eef_pose = fk[frame].squeeze()
# eef_pose = np.eye(4)
eef_fk = robot.eef_fk(eef_pose, frame)

lnk8_pose = eef_pose_to_link8(eef_pose, frame)
franka_eef_fk_arr = franka_eef_link_fk(prismatic_joint, lnk8_pose)
franka_eef_fk = {
    "panda_link8": franka_eef_fk_arr[0],
    "panda_hand": franka_eef_fk_arr[1],
    "panda_grasptarget": franka_eef_fk_arr[2],
    "right_gripper": franka_eef_fk_arr[3],
    "panda_leftfinger": franka_eef_fk_arr[4],
    "panda_rightfinger": franka_eef_fk_arr[5],
}
# visual_eef_fk = robot.eef_visual_fk(pose, frame, auxiliary_joint_values)

if np.allclose(eef_fk["panda_link8"], lnk8_pose):
    cprint("eef_fk['panda_link8'] == lnk8_pose", "green")

print("-" * 30)
faulty_links = []
for link_name in franka_eef_fk:
    if np.allclose(eef_fk[link_name], franka_eef_fk[link_name]):
        cprint(f"eef_fk['{link_name}'] EQUALS franka_eef_fk['{link_name}']", "cyan")
    else:
        cprint(f"eef_fk['{link_name}'] DOES NOT EQUAL franka_eef_fk['{link_name}']", "red")
        faulty_links.append(link_name)
    print("-" * 30)


eef_pts = get_points_on_robot_eef(robot, eef_pose, num_points=0, link_points=np_sampler.points, frame=frame)
franka_eef_pts = get_points_on_franka_eef(eef_pose, prismatic_joint=0.04, sample=0, 
                                          panda_hand_points=np_franka_sampler.points["eef_panda_hand"],
                                          panda_leftfinger_points=np_franka_sampler.points["eef_panda_leftfinger"],
                                          panda_rightfinger_points=np_franka_sampler.points["eef_panda_rightfinger"],
                                          frame=frame)


eef_fk['panda_link8'] == lnk8_pose
------------------------------
eef_fk['panda_link8'] EQUALS franka_eef_fk['panda_link8']
------------------------------
eef_fk['panda_hand'] EQUALS franka_eef_fk['panda_hand']
------------------------------
eef_fk['panda_grasptarget'] EQUALS franka_eef_fk['panda_grasptarget']
------------------------------
eef_fk['right_gripper'] EQUALS franka_eef_fk['right_gripper']
------------------------------
eef_fk['panda_leftfinger'] EQUALS franka_eef_fk['panda_leftfinger']
------------------------------
eef_fk['panda_rightfinger'] EQUALS franka_eef_fk['panda_rightfinger']
------------------------------


In [10]:
import viz_client

urdf_path = "/workspace/assets/panda/panda_spheres.urdf"
if not os.path.exists(urdf_path):
    print(f"❌ URDF not found at {urdf_path}")
    print("Please update the urdf_path variable in test_connect()")
    raise FileNotFoundError(f"URDF not found at {urdf_path}")

viz_client.connect(urdf_path)
viz_client.publish_joints(joints=robot.neutral_config_dict)

viz_server not running — starting…
Connected to viz_server


In [11]:
viz_client.publish_ghost_end_effector(pose=fk["panda_hand"], frame="panda_hand", color=[0.8, 0.2, 0.8])

In [12]:
viz_client.clear_ghost_end_effector()

In [13]:
config = robot.neutral_config.copy()
config[0] = robot.neutral_config[0] + 1.0 
viz_client.publish_ghost_robot(config, color=[0.5,0.6,0.8])

In [14]:
viz_client.clear_ghost_robot()

In [15]:
from robofin.old.robot_constants import FrankaConstants, RealFrankaConstants
print(FrankaConstants.JOINT_LIMITS[5])
print(RealFrankaConstants.JOINT_LIMITS[5])
print(robot.main_joint_limits[5])


[-0.018  3.752]
[0.5  3.75]
[-0.087  3.822]


## Dataloader testing

In [1]:
from avoid_everything.data_loader import DataModule

data_module_parameters = {
    "data_dir": "/workspace/datasets/ae_aristotle1_5mm_cubbies",
    "train_trajectory_key": "global_solutions",
    "val_trajectory_key": "global_solutions",
    "num_obstacle_points": 4096,
    "random_scale": 0.015
}
shared_parameters = {
    "urdf_path": "assets/panda/panda.urdf",
    "num_robot_points": 2048,
    "num_target_points": 128,
    "action_chunk_length": 1
}
cfg = {
    "train_batch_size": 10,
    "val_batch_size": 10,
    "num_workers": 4,
}

dm = DataModule(
    train_batch_size=cfg["train_batch_size"],
    val_batch_size=cfg["val_batch_size"],
    num_workers=cfg["num_workers"],
    **data_module_parameters,
    **shared_parameters,
)

dm.setup()

dl = dm.train_dataloader()
sample = dl.dataset[100]

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dl.dataset.__getitem__(3)["target_position"].shape

torch.Size([3])

In [ ]:
import numpy as np
import torch

def convert_to_numpy_f32(arr: np.ndarray | torch.Tensor) -> np.ndarray:
    """
    Convert a NumPy array or Torch tensor to a NumPy float32 array.
    
    Parameters
    ----------
    arr : np.ndarray or torch.Tensor
        Input array to convert.
    
    Returns
    -------
    np.ndarray
        Converted array with dtype float32.
    """
    if isinstance(arr, torch.Tensor):
        np_arr: np.ndarray = arr.cpu().numpy()
    elif isinstance(arr, np.ndarray):
        np_arr: np.ndarray = arr
    else:
        raise TypeError("convert_to_numpy_f32: Input must be a NumPy array or Torch tensor")
    return np_arr.astype(np.float32) 

In [ ]:
cuboid_dims = convert_to_numpy_f32(sample["cuboid_dims"])
cuboid_centers = convert_to_numpy_f32(sample["cuboid_centers"])
cuboid_quaternions = convert_to_numpy_f32(sample["cuboid_quats"])
for dims, center, quat in zip(cuboid_dims, cuboid_centers, cuboid_quaternions):
    print(f"Lengths: Dimensions: {len(dims)}, Center: {len(center)}, Quaternion: {len(quat)}")

cylinder_radii = convert_to_numpy_f32(sample["cylinder_radii"])
cylinder_heights = convert_to_numpy_f32(sample["cylinder_heights"])
cylinder_centers = convert_to_numpy_f32(sample["cylinder_centers"])
cylinder_quaternions = convert_to_numpy_f32(sample["cylinder_quats"])
for radius, height, center, quat in zip(cylinder_radii, cylinder_heights, cylinder_centers, cylinder_quaternions):
    print(f"Lengths: Radius: {len(radius)}, Height: {len(height)}, Center: {len(center)}, Quaternion: {len(quat)}")

In [ ]:
import viz_client

urdf_path = "/workspace/assets/panda/panda_spheres.urdf"
if not os.path.exists(urdf_path):
    print(f"❌ URDF not found at {urdf_path}")
    print("Please update the urdf_path variable in test_connect()")
    raise FileNotFoundError(f"URDF not found at {urdf_path}")

viz_client.connect(urdf_path)

In [ ]:
viz_client.publish_obstacles(cuboid_centers=cuboid_centers,
                            cuboid_dims=cuboid_dims,
                            cuboid_quaternions=cuboid_quaternions,
                            cylinder_centers=cylinder_centers,
                            cylinder_radii=cylinder_radii,
                            cylinder_heights=cylinder_heights,
                            cylinder_quaternions=cylinder_quaternions,
                            color=[0.8, 0.5, 0.6])

In [ ]:
viz_client.clear_obstacles()

In [ ]:
viz_client.publish_joints(joints=robot.neutral_config_dict)

In [ ]:
viz_client.shutdown()